In [1]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
import os
import warnings
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import xarray as xr
warnings.filterwarnings('ignore')
%run ./startup_file.ipynb #run startup file and install libraries if necessary

# SAT_SHIP_MATCHUP

## Author: haley synan

##### *Last run on 2024-07-23*

In [2]:
os.chdir(data_dir)
os.getcwd()

'C:\\users\\haley.synan\\Documents\\SEASCAPES\\DATA'

# Load ship track

In [ ]:
#df.UTC_DATETIME.str.find('2010-05-26')

In [3]:
#using ecomon data as sample for testing 
track_path = os.path.join(data_dir,
                          'data_2000_2005.csv')

df = pd.read_csv(track_path)
df = df[1:100000] #take only first 100 obs, 0th row is units (dont take that) 

df

,lat,lon,date,datetime,depth,temp,sal,source,SA,CT,rho,mlp2,spiciness0,alpha,beta,month,tempAnomalies,salAnomalies,year
1,41.97140,-66.07820,20010214,14-Feb-2001 14:45:59,1.984309,4.524,32.588,CTD,32.742733,4.544048,1025.826,33.099998,-1.296677,0.000102,0.000769,2,-0.756052,-0.158112,2001
2,41.97140,-66.07820,20010214,14-Feb-2001 14:45:59,2.976457,4.523,32.589,CTD,32.743729,4.542965,1025.831,33.099998,-1.295921,0.000102,0.000769,2,-0.757135,-0.157116,2001
3,41.97140,-66.07820,20010214,14-Feb-2001 14:45:59,3.968599,4.519,32.593,CTD,32.747742,4.538846,1025.839,33.099998,-1.293732,0.000102,0.000769,2,-0.761254,-0.153103,2001
4,41.97140,-66.07820,20010214,14-Feb-2001 14:45:59,4.960737,4.518,32.592,CTD,32.746735,4.537786,1025.843,33.099998,-1.294510,0.000102,0.000769,2,-0.762314,-0.154110,2001
5,41.97140,-66.07820,20010214,14-Feb-2001 14:45:59,5.952870,4.517,32.593,CTD,32.747734,4.536705,1025.849,33.099998,-1.294044,0.000102,0.000769,2,-0.763395,-0.153111,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,41.29167,-69.20333,20050519,19-May-2005 13:59:59,34.724800,5.360,31.882,CTD,32.033405,5.387768,1025.327,30.750967,-1.700706,0.000110,0.000766,5,-1.899988,-0.784098,2005
99996,41.29167,-69.20333,20050519,19-May-2005 13:59:59,35.716850,5.350,31.879,CTD,32.030403,5.377697,1025.331,30.750967,-1.704033,0.000110,0.000767,5,-1.910059,-0.787101,2005
99997,41.29167,-69.20333,20050519,19-May-2005 13:59:59,36.708890,5.350,31.878,CTD,32.029408,5.377634,1025.335,30.750967,-1.704895,0.000110,0.000766,5,-1.910122,-0.788096,2005
99998,41.29167,-69.20333,20050519,19-May-2005 13:59:59,37.700930,5.330,31.871,CTD,32.022385,5.357580,1025.336,30.750967,-1.712181,0.000110,0.000767,5,-1.930176,-0.795119,2005


In [4]:
# divide datetime to year, month, day columns
#yr = []
#month =[]
#day = []
#for x in range(1,len(df.UTC_DATETIME)+1):
#    d = str(df.UTC_DATETIME[x]).split('-')
#    yr.append(d[0])
#    month.append(d[1])
#    d = d[2].split('T')
#    day.append(d[0])

#df.insert(2, "Year", yr, True)
#df.insert(3, "Month", month, True)
#df.insert(4, "Day", day, True)

# divide datetime to year, month, day columns
#dt = []
#for x in range(1,len(df.datetime)+1):
    #d = str(df.UTC_DATETIME[x]).split('T')
    #dt.append(d[0] + 'T00:00:00.000000000')
    #d = d[0] + 'T00:00:00.000000000'
    #d = str(df.date[x])+ 'T00:00:00.000000000'
    #dt.append(np.datetime64(d))
#df.insert(2, 'time', dt, True)

dt = []
for x in range(1,len(df.datetime)+1):
    d = str(df.datetime[x])
    splitat = 2
    d = d[:splitat]
    dt.append(d)

df.insert(2, 'day', dt, True)

In [5]:
ddt = []
for x in range(1,len(df.datetime)+1):
    dd = str(df.year[x])+'-0'+str(df.month[x])+'-'+str(df.day[x])+'T00:00:00.000000000'
    ddt.append(dd)

df.insert(2, 'time',ddt,True)
    

In [6]:
df.rename(columns={'lat': 'latitude', 'lon': 'longitude'}, inplace=True)

In [7]:
df[~df.index.duplicated()]

,latitude,longitude,time,day,date,datetime,depth,temp,sal,source,...,CT,rho,mlp2,spiciness0,alpha,beta,month,tempAnomalies,salAnomalies,year
1,41.97140,-66.07820,2001-02-14T00:00:00.000000000,14,20010214,14-Feb-2001 14:45:59,1.984309,4.524,32.588,CTD,...,4.544048,1025.826,33.099998,-1.296677,0.000102,0.000769,2,-0.756052,-0.158112,2001
2,41.97140,-66.07820,2001-02-14T00:00:00.000000000,14,20010214,14-Feb-2001 14:45:59,2.976457,4.523,32.589,CTD,...,4.542965,1025.831,33.099998,-1.295921,0.000102,0.000769,2,-0.757135,-0.157116,2001
3,41.97140,-66.07820,2001-02-14T00:00:00.000000000,14,20010214,14-Feb-2001 14:45:59,3.968599,4.519,32.593,CTD,...,4.538846,1025.839,33.099998,-1.293732,0.000102,0.000769,2,-0.761254,-0.153103,2001
4,41.97140,-66.07820,2001-02-14T00:00:00.000000000,14,20010214,14-Feb-2001 14:45:59,4.960737,4.518,32.592,CTD,...,4.537786,1025.843,33.099998,-1.294510,0.000102,0.000769,2,-0.762314,-0.154110,2001
5,41.97140,-66.07820,2001-02-14T00:00:00.000000000,14,20010214,14-Feb-2001 14:45:59,5.952870,4.517,32.593,CTD,...,4.536705,1025.849,33.099998,-1.294044,0.000102,0.000769,2,-0.763395,-0.153111,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,41.29167,-69.20333,2005-05-19T00:00:00.000000000,19,20050519,19-May-2005 13:59:59,34.724800,5.360,31.882,CTD,...,5.387768,1025.327,30.750967,-1.700706,0.000110,0.000766,5,-1.899988,-0.784098,2005
99996,41.29167,-69.20333,2005-05-19T00:00:00.000000000,19,20050519,19-May-2005 13:59:59,35.716850,5.350,31.879,CTD,...,5.377697,1025.331,30.750967,-1.704033,0.000110,0.000767,5,-1.910059,-0.787101,2005
99997,41.29167,-69.20333,2005-05-19T00:00:00.000000000,19,20050519,19-May-2005 13:59:59,36.708890,5.350,31.878,CTD,...,5.377634,1025.335,30.750967,-1.704895,0.000110,0.000766,5,-1.910122,-0.788096,2005
99998,41.29167,-69.20333,2005-05-19T00:00:00.000000000,19,20050519,19-May-2005 13:59:59,37.700930,5.330,31.871,CTD,...,5.357580,1025.336,30.750967,-1.712181,0.000110,0.000767,5,-1.930176,-0.795119,2005


# Convert dataframe to xarray dataset

In [ ]:
# THIS WORKS 
ds_insitu = df.set_index(['depth','latitude', 'time', 'longitude']).to_xarray() #set indexes and convert
#ds_insitu = df.set_index(['longitude']).to_xarray()

In [ ]:
#convert to pixel space 
#import xarray_regrid
ds_is = ds_insitu.regrid.linear(ds)

In [ ]:
ds_insitu

# Load satellite data 

In [ ]:
erddap_url = '/'.join(['https://coastwatch.pfeg.noaa.gov',
                       'erddap',
                       'griddap',
                       'pmlEsaCCI60OceanColor8Day'])

ds = xr.open_dataset(erddap_url)
ds

In [ ]:
print('Temporal and spatial ranges of the satellite dataset')
print('time range', ds.attrs['time_coverage_start'], 
      ds.attrs['time_coverage_end'])
print('latitude range', ds.attrs['geospatial_lat_min'], 
      ds.attrs['geospatial_lat_max'])
print('longitude range', ds.attrs['geospatial_lon_min'], 
      ds.attrs['geospatial_lon_max'])
print(' ')
print('Temporal and spatial ranges of the track data')
print('time range', df.UTC_DATETIME.min(), df.UTC_DATETIME.max())
print('latitude range', 
      df.latitude.min(), df.latitude.max())
print('longitude range', 
      df.longitude.min(), df.longitude.max())

In [ ]:
# Create a temporary Pandas data frame to hold the downloaded satellite data
col_names = ["erddap_date", "matched_lat", "matched_lon", "matched_chla"]
tot = pd.DataFrame(columns=col_names)

# Finish each URL and download
for i in range(1, len(df)+1):
    clear_output(wait=True)
    print(i+1, 'of', len(df))
    
    # Crop the dataset to include data that corresponds to track locations
    cropped_ds = ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i],
                                   longitude=df.longitude[i],
                                   method='nearest'
                                   )
     
    # Downloaded the data and add it to a new line in the tot data frame
    tot.loc[len(tot.index)] = [cropped_ds.time.values,
                               np.round(cropped_ds.latitude.values, 5),  # round 5 dec
                               np.round(cropped_ds.longitude.values, 5), # round 5 dec
                               np.round(cropped_ds.values, 2)  # round 2 decimals
                               ]
    
    print(tot.loc[[len(tot)-1]])

tot.head(2)

In [ ]:
df[['matched_lat', 
    'matched_lon', 
    'matched_chla', 
    'erddap_date']] = tot[['matched_lat',
                           'matched_lon',
                           'matched_chla',
                           'erddap_date']]

df

In [ ]:
os.chdir(data_dir)
df.to_csv('chl_matchup.csv', index=False, encoding='utf-8')

In [ ]:
###############################################

In [ ]:
lat = np.transpose(df.latitude)
lon = np.transpose(df.longitude)
d = np.column_stack((lat,lon))

#find location of matched lats lons

In [ ]:
# Create a temporary Pandas data frame to hold the downloaded satellite data
col_names = ["erddap_date", "matched_lat", "matched_lon", "matched_chla"]
tot = pd.DataFrame(columns=col_names)

# Finish each URL and download
for i in range(1, len(df)+1):
    clear_output(wait=True)
    print(i, 'of', len(df)+1)
    d1= []
    # Crop the dataset to include data that corresponds to track locations
    d1= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i],
                                   longitude=df.longitude[i],
                                   method='nearest'
                                   )
     # Crop the dataset to include data that corresponds to track locations
    d2= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i]-1,
                                   longitude=df.longitude[i]-1,
                                   method='nearest'
                                   )

 # Crop the dataset to include data that corresponds to track locations
    d3=  ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i]-1,
                                   longitude=df.longitude[i],
                                   method='nearest'
                                   )
     # Crop the dataset to include data that corresponds to track locations
    d4= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i]-1,
                                   longitude=df.longitude[i]+1,
                                   method='nearest'
                                   )
     # Crop the dataset to include data that corresponds to track locations
    d5= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i],
                                   longitude=df.longitude[i]-1,
                                   method='nearest'
                                   )

     # Crop the dataset to include data that corresponds to track locations
    d6= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i],
                                   longitude=df.longitude[i]+1,
                                   method='nearest'
                                   )
     # Crop the dataset to include data that corresponds to track locations
    d7= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i]+1,
                                   longitude=df.longitude[i]-1,
                                   method='nearest'
                                   )
     # Crop the dataset to include data that corresponds to track locations
    d8= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i]+1,
                                   longitude=df.longitude[i],
                                   method='nearest'
                                   )
     # Crop the dataset to include data that corresponds to track locations
    d9= ds['chlor_a'].sel(time=df.time[i],
                                   latitude=df.latitude[i]+1,
                                   longitude=df.longitude[i]+1,
                                   method='nearest'
                                   )

    chl_avg = np.nanmean([d1.values, d2.values, d3.values, d4.values, d5.values, d6.values, d7.values, d8.values, d9.values])

 # Downloaded the data and add it to a new line in the tot data frame
    tot.loc[len(tot.index)] = [df.time[i],
                               np.round(df.latitude[i], 5),  # round 5 dec
                               np.round(df.longitude[i], 5), # round 5 dec
                               np.round(chl_avg, 5) 
                               ]
     
